In [ ]:

import language_tool_python

# Ładujemy model językowy dla języka polskiego w spaCy
nlp = spacy.load('pl_core_news_sm')

# Funkcja do sprawdzania błędów i sugerowania poprawek
def popraw_tekst(tekst):
    # Używamy LanguageTool do wykrywania błędów
    tool = language_tool_python.LanguageTool('pl')
    matches = tool.check(tekst)
    
    # Poprawiamy błędy
    poprawiony_tekst = language_tool_python.utils.correct(tekst, matches)
    
    # Dodatkowo używamy spaCy do analizy i usuwania zbędnych znaków
    doc = nlp(poprawiony_tekst)
    poprawiony_tekst_spacy = " ".join([token.text for token in doc])
    
    return poprawiony_tekst_spacy

# Tekst wejściowy
tekst = "Cześc, jesem Marcni — co tam u czi sychać!"

# Poprawiamy tekst
poprawiony_tekst = popraw_tekst(tekst)
print("Poprawiony tekst:", poprawiony_tekst)


ModuleNotFoundError: No module named 'spacy'

In [6]:
from transformers import BertForMaskedLM, BertTokenizer
import torch

# Ładowanie tokenizera i modelu Polbert (BERT przeszkolony na języku polskim)
model_name = "dkleczek/bert-base-polish-uncased-v1"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)

# Funkcja do poprawy tekstu
def popraw_tekst(tekst):
    # Tokenizowanie tekstu
    inputs = tokenizer(tekst, return_tensors="pt")
    # Predykcja z modelu
    with torch.no_grad():
        logits = model(**inputs).logits
    
    # Decodowanie wyników
    predicted_token_ids = torch.argmax(logits, dim=-1)
    corrected_tekst = tokenizer.decode(predicted_token_ids[0], skip_special_tokens=True)
    
    return corrected_tekst

# Tekst wejściowy
tekst = "Cześć, jestem Marcni — co tam u czbiy sychać!"

# Poprawiamy tekst
poprawiony_tekst = popraw_tekst(tekst)
print("Poprawiony tekst:", poprawiony_tekst)


Some weights of the model checkpoint at dkleczek/bert-base-polish-uncased-v1 were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Poprawiony tekst: ! czesc, jestem marcni — co tam u czbiy sychac!.


In [13]:
import language_tool_python

tool = language_tool_python.LanguageTool('pl')

text = "Cześć, jestem Marcni— co tam u czbiy sychać!"

matches = tool.check(text)
for _ in matches:
    print(_)

corrected_text = language_tool_python.utils.correct(text, matches)

print(corrected_text)


Offset 14, length 6, Rule ID: MORFOLOGIK_RULE_PL_PL
Message: Wykryto prawdopodobny błąd pisowni
Suggestion: Marcin; Marini; Marci; Marani; Maroni; Marcini; Marconi; Maryni; Marcuj; Marni; Mar cni; Marc ni
Cześć, jestem Marcni— co tam u czbiy sychać!
              ^^^^^^
Offset 31, length 5, Rule ID: MORFOLOGIK_RULE_PL_PL
Message: Wykryto prawdopodobny błąd pisowni
Cześć, jestem Marcni— co tam u czbiy sychać!
                               ^^^^^
Offset 37, length 6, Rule ID: MORFOLOGIK_RULE_PL_PL
Message: Wykryto prawdopodobny błąd pisowni
Suggestion: słychać; spychać; dychać; osychać; sycząc; sycąc; usychać; zsychać
Cześć, jestem Marcni— co tam u czbiy sychać!
                                     ^^^^^^
Cześć, jestem Marcin— co tam u czbiy słychać!


In [17]:
import torch
from gramformer import Gramformer

def popraw_zdanie(zdanie):
    # Sprawdzamy, czy dostępny jest GPU, jeśli nie, używamy CPU
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    # Inicjalizacja Gramformer
    gf = Gramformer(models=1, device=device)  # Używamy modelu do poprawy gramatyki

    # Poprawa zdania
    poprawione_zdanie = gf.correct(zdanie)
    
    return poprawione_zdanie

# Przykładowe zdanie z błędami
zdanie = "Cześć, jestem Marcin— co tam u czbiy słychać! Jak tam twoj dzien?"

# Wywołanie funkcji
poprawione_zdanie = popraw_zdanie(zdanie)
print(poprawione_zdanie)

ModuleNotFoundError: No module named 'gramformer'

In [ ]:
import language_tool_python
from transformers import pipeline
import time
# tool = language_tool_python.LanguageTool('pl')

# google-bert/bert-base-multilingual-uncased
# dkleczek/bert-base-polish-uncased-v1
# mlx-community/bert-base-multilingual-uncased
# lxyuan/span-marker-bert-base-multilingual-uncased-multinerd
fill_mask = pipeline("fill-mask", model="Geotrend/bert-base-pl-cased")

text = "Cześć, jestem Marcni — co tam u czbiy sychać!"
text = "Cześć, jestem Marcin, co ta mu ciebie słyszać mój drogi kolego."
time1 = time.time()
matches = tool.check(text)
print(time.time() - time1)


unresolved_words = []
for match in matches:
    if not match.replacements:  # No suggestions
        start, end = match.offset, match.offset + match.errorLength
        unresolved_words.append((text[start:end], start, end))

for word, start, end in unresolved_words:
    time1 = time.time()
    masked_text = corrected_text[:start] + "[MASK]" + corrected_text[end:]
    predictions = fill_mask(masked_text)
    
    best_suggestion = predictions[0]['token_str']
    corrected_text = corrected_text[:start] + best_suggestion + corrected_text[end:]
    print(time.time() - time1)
print("Original Text:", text)
print("Corrected Text:", corrected_text)


c:\Users\kogut\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


0.05855393409729004
Original Text: Cześć, jestem Marcin, co ta mu ciebie słyszać mój drogi kolego.
Corrected Text: Cześć, jestem Marcin, co ta mu ciebie słysząc mój drogi kolego.


In [40]:
import language_tool_python
import time

# tool = language_tool_python.LanguageTool('pl-PL')

# google-bert/bert-base-multilingual-uncased
# dkleczek/bert-base-polish-uncased-v1
# mlx-community/bert-base-multilingual-uncased
# lxyuan/span-marker-bert-base-multilingual-uncased-multinerd
# Geotrend/bert-base-pl-cased   - od
fill_mask = pipeline("fill-mask", model="google-bert/bert-base-multilingual-uncased")

def check_text(text):
    matches = tool.check(text)
    print("Oryginalny tekst:", text)
    
    unresolved_words = []
    
    if matches:
        for match in matches:
            start, end = match.offset, match.offset + match.errorLength
            error_word = text[start:end]
            unresolved_words.append((error_word, start, end))
            # print(f"Błąd: '{error_word}' w pozycji {start}-{end} - {match.message}")
    else:
        print("Brak błędów.")
    
    corrected_text = text
    # print(unresolved_words)
    for word, start, end in unresolved_words:
        time1 = time.time()
        masked_text = corrected_text[:start] + "[MASK]" + corrected_text[end:]
        print("Masked text:", masked_text)
        # Tutaj można dodać funkcję do przewidywania poprawnych słów
        predictions = fill_mask(masked_text)  # Załóżmy, że fill_mask jest wcześniej zdefiniowaną funkcją
        
        best_suggestion = predictions[0]['token_str']
        corrected_text = corrected_text[:start] + best_suggestion + corrected_text[end:]
        print("Czas przetwarzania:", time.time() - time1)
    
    print("Poprawiony tekst:", corrected_text)

text = "Cześć, jestem Marcin, co ta mu ciebie słyszać mój drogi kolego."
text = "Cześć, nazywam się Kuba, jestem zawodnikiem Lechich orzów, Lech jak tańsk to kurwy."
check_text(text)

c:\Users\kogut\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at google-bert/bert-base-multilingual-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSeq

Oryginalny tekst: Cześć, nazywam się Kuba, jestem zawodnikiem Lechich orzów, Lech jak tańsk to kurwy.
Masked text: Cześć, nazywam się Kuba, jestem zawodnikiem [MASK] orzów, Lech jak tańsk to kurwy.
Czas przetwarzania: 0.15599894523620605
Masked text: Cześć, nazywam się Kuba, jestem zawodnikiem klubu or[MASK]Lech jak tańsk to kurwy.
Czas przetwarzania: 0.16199970245361328
Masked text: Cześć, nazywam się Kuba, jestem zawodnikiem klubu or##łaLech jak tań[MASK] kurwy.
Czas przetwarzania: 0.14300179481506348
Poprawiony tekst: Cześć, nazywam się Kuba, jestem zawodnikiem klubu or##łaLech jak tań##ce kurwy.


In [6]:
# import language_tool_python
# tool = language_tool_python.LanguageTool('pl')

# Przykładowy tekst
text = "Cześć, jestem Marcni — co tam u czbiy sychać!"
text = "Cześć, jestem Marcin, co ta mu ciebie słyśac mój drogi kolego."
# Sprawdzanie tekstu przy użyciu LanguageTool
matches = tool.check(text)

# Wypisanie oryginalnego tekstu
print("Oryginalny tekst:", text)

# Wypisanie błędów oraz ich lokalizacji
if matches:
    for match in matches:
        start, end = match.offset, match.offset + match.errorLength
        error_word = text[start:end]
        print(f"Błąd: '{error_word}' w pozycji {start}-{end} - {match.message}")
else:
    print("Brak błędów w tekście.")

Oryginalny tekst: Cześć, jestem Marcin, co ta mu ciebie słyśac mój drogi kolego.
Błąd: 'słyśac' w pozycji 38-44 - Wykryto prawdopodobny błąd pisowni


In [11]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Load a smaller model and tokenizer
model_name = "facebook/distilbart-cnn-12-6"  # Adapt this if an appropriate Polish model is available
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Example input sentence to correct
input_text = "Cześć, nazywam się Kuba, jestem zawodnikiem Lechich orzów, Lech jak tańsk to klub."

# Prepare the input
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=512)

# Generate corrected text
with torch.no_grad():  # Disabling gradients
    generated_ids = model.generate(inputs['input_ids'], max_length=50)

# Decode the generated text
corrected_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

# Output the original and the corrected text
print("Original Sentence:", input_text)
print("Corrected Sentence:", corrected_text)

OSError: facebook/distilbart-cnn-12-6 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`